# Imports

- Importando as bibliotecas necessárias:
    - `pandas` é importado para manipulação de dados.
    - `matplotlib.pyplot` é importado para visualização de dados.
    - `seaborn` é importado para visualização estatística de dados.
    - `train_test_split` é importado de `sklearn.model_selection` para dividir os dados em conjuntos de treinamento e teste.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
from torchvision import transforms

# Análise exploratória de dados

Antes de definir o que iremos fazer sobre os dados, será feito uma análise exploratória inicial. Para isso, vamos entender melhor os dados.
- Carregando os dados:
    - `pd.read_csv()` é usado para ler o arquivo CSV que contém os dados.
    - O caminho para o arquivo CSV é "/kaggle/input/crop-diseases-classification/Data/train.csv".
    - Os dados são armazenados em um DataFrame chamado `dsDiseases`.

In [ ]:
dsDiseases = pd.read_csv("/kaggle/input/crop-diseases-classification/Data/train.csv")

Nessa primeira parte, vamos entender o que é cada coluna:
- Examinando as primeiras linhas do DataFrame:
    - `head()` é usado para visualizar as primeiras linhas do DataFrame `dsDiseases`.
    - Isso ajuda a ter uma ideia inicial da estrutura e dos dados presentes no DataFrame.

In [ ]:
dsDiseases.head()

Para uma visão ampla em relação aos dados, podemos plotar dois gráficos: um gráfico em barras (com a contagem de cada categoria) e um gráfico em pizza (com a porcentagem das categorias).
- Contagem e visualização das categorias:
    - `value_counts()` é usado para contar o número de ocorrências de cada categoria na coluna 'label' do DataFrame `dsDiseases`.
    - Os resultados são armazenados na variável `counts`.
    - Um gráfico de barras é criado usando `sns.barplot()` para visualizar a contagem de cada categoria.
    - `counts.index` é usado como os valores no eixo x (categorias).
    - `counts.values` é usado como os valores no eixo y (contagem).
    - O título do gráfico é definido como 'Crop diseases'.
    - Os rótulos dos eixos x e y são definidos como 'Categorias' e 'Contagem', respectivamente.
    - `plt.show()` é usado para exibir o gráfico.

In [ ]:
# Contagem das categorias
counts = dsDiseases['label'].value_counts()

# Plot
sns.barplot(x=counts.index, y=counts.values)
plt.title('Contagem de categorias')
plt.xlabel('Categorias')
plt.ylabel('Contagem')
plt.show()

- Visualização das categorias em um gráfico de pizza:
    - `plt.pie()` é usado para criar um gráfico de pizza com as porcentagens de cada categoria.
    - `counts` é passado como os dados para o gráfico de pizza.
    - `labels=counts.index` é usado para rotular as fatias do gráfico de acordo com as categorias.
    - `autopct='%1.1f%%'` formata as porcentagens exibidas em cada fatia.
    - `startangle=140` define o ângulo inicial da pizza para uma melhor visualização.
    - `plt.title()` define o título do gráfico como 'Crop diseases'.
    - `plt.axis('equal')` garante que a pizza seja desenhada como um círculo.
    - `plt.show()` é usado para exibir o gráfico de pizza.

In [ ]:
# Plot
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Crop diseases')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

# Divisão de treino, teste e validação

Bom, após vizualizarmos ambos os gráficos, percebemos que está desbalanceado, pois a categoria 3 possui 61.5% de amostras, o que é um valor maior do que a soma de todas as outras categorias, isso para um dataset não é bom, por isso vamos para a etapa de Divisão de treino, teste e validação de dados para o nosso dataset.

In [ ]:
# Divisão dos dados em conjuntos de treinamento e teste, mantendo as amostras proporcionais
train_data, test_data = train_test_split(dsDiseases, test_size=0.2, random_state=42, stratify=dsDiseases['label'])

In [ ]:
# Plot dos dados de treinamento
sns.barplot(x=train_data['label'].value_counts().index, y=train_data['label'].value_counts().values)
plt.title('Contagem de Categorias - Conjunto de Treinamento')
plt.xlabel('Categorias')
plt.ylabel('Contagem')
plt.show()

In [ ]:
# Plot dos dados de teste
sns.barplot(x=test_data['label'].value_counts().index, y=test_data['label'].value_counts().values)
plt.title('Contagem de Categorias - Conjunto de Teste')
plt.xlabel('Categorias')
plt.ylabel('Contagem')
plt.show()

# Tratamento de dados

Técnicas de pré processamento acontecem dentro do dataset, antes de passar para o modelo, para que o dataset fique balanceado e tanto ele quanto o resultado seja justo.

Task:
Objetivo: Deixar todas as classes com a mesma quantidade. Aqui iremos deixar todas com o mesmo valor da classe 2, por ser a mediana.

- Classe 3 e 4: Recorte de dataset
    Será eliminado, de maneira aleatória, (classe3 - classe2) quantidades
    Motivo: Para diminuir a quantidade
- Classes 0 e 1: Data augmentation
    Motivo: Todas essas duas classes possuem uma amostragem muito menor. Sendo assim, iremos aumentar, através de técnicas de rotação e iluminação, os dados dessas classes. Com isso, teremos a mesma quantidade de dados da classe 2

In [ ]:
# Definindo as transformações para data augmentation
data_augmentation = transforms.Compose([
    transforms.RandomRotation(degrees=30),  # rotação aleatória de até 30 graus
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # ajustes de cor aleatórios
])

# Recorte de dataset para classes 3 e 4
def crop_dataset(dataset, target_class, target_size):
    indices_to_keep = [idx for idx, label in enumerate(dataset.targets) if label != target_class]
    cropped_dataset = torch.utils.data.Subset(dataset, indices_to_keep[:target_size])
    return cropped_dataset

# Aplicando recorte para classes 3 e 4
cropped_dataset_3 = crop_dataset(original_dataset, target_class=3, target_size=desired_size)
cropped_dataset_4 = crop_dataset(original_dataset, target_class=4, target_size=desired_size)

# Aplicando transformações de data augmentation para classes 0 e 1
def augment_dataset(dataset, target_classes, target_size, augmentation_transform):
    augmented_data = []
    for idx, (image, label) in enumerate(dataset):
        if label in target_classes:
            # Aplicando a transformação de data augmentation
            augmented_data.append((augmentation_transform(image), label))
            if len(augmented_data) >= target_size:
                break
    return augmented_data

# Aumentando dados para classes 0 e 1
augmented_data_0 = augment_dataset(original_dataset, target_classes=[0], target_size=desired_size, augmentation_transform=data_augmentation)
augmented_data_1 = augment_dataset(original_dataset, target_classes=[1], target_size=desired_size, augmentation_transform=data_augmentation)

# Concatenando os datasets resultantes
balanced_dataset = torch.utils.data.ConcatDataset([cropped_dataset_3, cropped_dataset_4, augmented_data_0, augmented_data_1])

# Normalização de dados

# Escolha modelo

# Treinamento

# Avaliação do modelo